# Lineare Regression

Lineare Regressionsmodelle sind ein guter Ausgangspunkt für Regressionsaufgaben.

Solche Modelle sind beliebt, weil sie sehr schnell angepasst werden können und sehr gut interpretierbar sind. 

Die einfachste Form eines linearen Regressionsmodells ist die Anpassung einer geraden Linie an Daten, wir werden nachher aber auch sehen, wie man kompliziertere Modelle anpassen kann.

Wir beginnen mit den Standard-Importen:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

plt.rcParams["figure.figsize"] = (10,8)

## Einfache lineare Regression

Eine Gerade ist durch die Gleichung

$y = b + ax$

gegeben, wobei $a$ die Steigung der Gerade ist und $b$ als y-Achsenabschnitt bezeichnet wird.

Generieren wir uns ein paar Daten, die um die Gerade $y = -5 + 2x$ gestreut sind:

> NumPys `np.ramdom.default_rng` gibt einen Pseudo-Zufallsgenerator zurück. Damit wir bei unseren Experimenten
> nachvollziehbare Werte bekommen, initiieren wir ihn mit einem festen `seed` - hier 1.
>
> `random` gibt Werte im Bereich $[0, 1)$ zurück. Mit `random(100)` erhalten wir ein Array von 100 Zufallswerten.
>
> `standard_normal` gibt analog Werte zurück, allerdings diesmal Standard-Normalverteilt. `random` ist gleichverteilt.

**Beachte:** Obwohl `x` ein Array ist, können wir damit in normaler "skalarer" Schreibweise `y` als Array errechnen.

In [ ]:
m = 100

rng = np.random.default_rng(1)
x = 10 * rng.random(m)
y = 2 * x - 5 + rng.standard_normal(m)
sns.scatterplot(x, y);

Wir können Scikit-Learns `LinearRegression Estimator` verwenden, um die beste Gerade zu ermitteln:

> `np.linspace` erzeugt ein "linear verteiltes" Array. D.h. `np.linspace(0, 10, 1000)` verteilt ermittelt 1000 gleichweit verteilte Punkte zwischen 0 und 10.

> `np.newaxis` erhöht die Dimension eines Vektors oder einer Matrix: aus einer ein-dimensionalen Struktur wird eine zwei-Dimensionale etc.


> `x[:,np.newaxis]` erzeugt aus dem Vektor (1D) $[x^{(1)}, x^{(2)}, \dots, x^{(n)}]$ eine Matrix (2D) $[[x^{(1)}], [x^{(2)}], \dots, [x^{(n)}]]$ - unsere Design-Matrix.

In [ ]:
v = 10 * rng.random(5)

print('Ursprüngliches Array v: {} - {}'.format(v.shape, v))
print('Nach v[:, np.newaxis]: {} - \n{}'.format(v[:, np.newaxis].shape, v[:, np.newaxis]))
print('Nach v[np.newaxis, :]: {} - {}'.format(v[np.newaxis, :].shape, v[np.newaxis, :]))

`variable.shape()` ist dein Freund: Es hilft regelmäßig zu prüfen, ob die Dimensionen der Matrizen tatsächlich zusammen passen

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

model.fit(x[:, np.newaxis], y)

xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

sns.scatterplot(x, y)
sns.lineplot(xfit, yfit);


Die Steigung und der y-Achsenabschnitt sind in den Anpassungsparametern des
Modells enthalten. Diese sind in Scikit-Learn immer durch einen abschließenden Unterstrich
gekennzeichnet. Hier sind es die Parameter `coef_` und `intercept_`:

In [ ]:
print("Modell: y = {} + {} * x".format(model.intercept_, model.coef_[0]))

Wir sehen, dass die Ergebnisse sehr nahe an den Inputs liegen.

Der LinearRegression-Schätzer ist jedoch noch viel leistungsfähiger.
Er kann neben einfachen Geradengleichungen auch mehrdimensionale lineare Modelle
der Form

$y=a_0+a_1x_1+a_2x_2+\dots$

mit höher-dimensionalen x ermitteln.

Geometrisch entspricht dies der Anpassung einer Ebene an Punkte in drei Dimensionen
oder der Anpassung einer Hyperebene an Punkte in höheren Dimensionen.

Die mehrdimensionale Natur solcher Regressionen macht es schwieriger, sie zu
visualisieren, aber wir können ja einfach mal testen, ob die lineare Regression die Parameter für Eingabedaten
**ohne Fehler** ermittelt:

In [ ]:
rng = np.random.default_rng(1)
X = rng.random((4, 3))
y = 0.5 + np.dot(X, [1.5, -2., 1.])

model.fit(X, y)
print(model.intercept_)
print(model.coef_)

Wir ermitteln $y$ als Punkte der Hyperebene, die durch die vier zufälligen Stützpunkte $x$ definiert sind, und die lineare
Regression gewinnt die Koeffizienten zurück, die zur Konstruktion der Hyperebene verwendet wurden.

> **Beachte:** Oben hatten wir Skalare mit einem Array multipliziert - dies können wir in Python mit dem gewöhnlichen Multiplikations-Operator tun. Hier multiplizieren wir zwei Vektoren: $(3,1)$ mal $(1,3)$ - dafür brauchen wir die [Matrizenmultiplikation (dot-Produkt)](https://de.wikipedia.org/wiki/Matrizenmultiplikation)

## Lineare Regression für nicht-lineare Daten?

Was aber tun, wenn die Daten **nicht** durch ein lineares Modell abbildbar sind?

Ein Trick, mit dem Sie die lineare Regression an nicht-lineare Beziehungen zwischen
Variablen anpassen können, besteht darin, die Daten mit sogenannten **Basisfunktionen**
zu transformieren.

Die Idee besteht darin, unser mehrdimensionales lineares Modell zu nehmen

$y=a_0+a_1x_1+a_2x_2+a_3x_3+\dots$

und mittels Basisfunktionen zu transformieren: Dazu lassen wir

$x_i = f_i(x)$ für i=1...k

sein, wobei $f_i$ eine Funktion ist, die unsere Daten transformiert.

Wenn zum Beispiel $f_i(x) = x^i$ ist, wird unser Modell zu einer Polynom-Regression:

$y=a_0+a_1x+a_2x^2+a_3x^3+\dots$

Beachten Sie, dass es sich nach wie vor um ein lineares Modell handelt - die
Linearität bezieht sich auf die Tatsache, dass die Koeffizienten $a_i$ sich niemals
miteinander multiplizieren oder dividieren. Was wir effektiv getan haben, ist,
unsere $x$-Werte in eine höhere Dimension zu projizieren, so
dass eine lineare Anpassung kompliziertere Beziehungen zwischen $x$ und $y$
einpassen kann.

## Polynomische Basisfunktionen

Diese Polynomprojektion ist so nützlich, dass sie mit Hilfe des
`PolynomialFeatures-Transformators` in Scikit-Learn eingebaut ist:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

Wir sehen hier, dass der Transformator unseren skalaren Wert in ein
Array mit drei Elementen umgewandelt hat, indem er Quadrat und dritte Potenz unseres Wertes
als "Feature" erzeugt hat.

Diese neue, höherdimensionale Input-Darstellung kann dann in eine lineare Regression
gesteckt werden. Der sauberste Weg, dies zu erreichen, ist die Verwendung einer sogenannten `Pipeline`.

Erstellen wir auf diese Weise ein Polynom-Modell 7. Grades:

In [ ]:
from sklearn.pipeline import make_pipeline
poly_model = make_pipeline(PolynomialFeatures(7), LinearRegression())

Mit dieser Transformation können wir das lineare Modell verwenden, um viel
kompliziertere Beziehungen zwischen $x$ und $y$ anzupassen. Hier ist zum Beispiel
eine polynomiale Regression einer verrauschte Sinuswelle:

In [ ]:
rng = np.random.default_rng(1)
x = 10 * rng.random(50)
y = np.sin(x) + 0.1 * rng.standard_normal(50)

poly_model.fit(x[:, np.newaxis], y)
yfit = poly_model.predict(xfit[:, np.newaxis])
sns.scatterplot(x, y)
sns.lineplot(xfit, yfit);

## Gaußsche Basisfunktionen

Natürlich sind auch andere Basisfunktionen möglich. Ein nützliches Muster ist zum
Beispiel die Anpassung eines Modells, das nicht eine Summe von Polynom-Basen,
sondern eine Summe von Gaußschen Basen ist.
Eine Gaußsche Basis sieht folgendermaßen aus:

In [ ]:
from scipy.stats import norm

x_all = np.arange(-10, 10, 0.001)
y2 = norm.pdf(x_all,0,1)

fig, ax = plt.subplots(figsize=(9,6))

sns.lineplot(x_all,y2, ax=ax)
ax.set_xlim([-4,4])
ax.set_yticklabels([])
ax.set_title('Gauß- / Normal-Verteilung');

 Diese Gaußschen Basisfunktionen sind nicht in Scikit-Learn eingebaut, aber wir
 können einen benutzerdefinierten Transformator schreiben, der sie erzeugt.

 Hier ermöglichen wir also einen Fit der Daten mithilfe eine Summe von (im Beispiel 20) Gauß-Verteilungen:

In [ ]:
import Gauss

gauss_model = make_pipeline(Gauss.GaussianFeatures(20),
                            LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

sns.scatterplot(x, y)
sns.lineplot(xfit, yfit);

Wir führen dieses Beispiel hier nur an, um deutlich zu machen, dass polynome
Basisfunktionen nichts Magisches an sich haben: Wenn Sie eine Art Intuition in den
Generierungsprozess Ihrer Daten haben, die Sie glauben lässt, dass die eine oder
andere Basis angemessen sein könnte, können Sie sie ebenfalls verwenden.
